# <p style="text-align: center;, font-style: strong;">Anonymization with Tensorflow Scala</p>


*Only for RVB image, small changes are necessary if you need to use it for grascales image*

## Dependencies

In [ ]:
interp.load.ivy("com.github.haifengl" % "smile-scala_2.12" % "1.5.2")

In [ ]:
interp.load.ivy(
  coursier.Dependency(
    module = coursier.Module("org.platanios", "tensorflow_2.12"),
    version = "0.4.1",
    // replace with linux-gpu-x86_64 on linux with nvidia gpu or with darwin-cpu-x86_64 on macOS 
    attributes = coursier.Attributes("", "darwin-cpu-x86_64")
  )
)
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

interp.load.ivy(
  coursier.Dependency(
    module = coursier.Module(coursier.Organization("org.platanios"), coursier.ModuleName("tensorflow_2.12")),
    version = "0.4.1",
    // replace with linux-gpu-x86_64 on linux with nvidia gpu or with darwin-cpu-x86_64 on macOS 
    attributes = coursier.Attributes(coursier.Type(""), coursier.Classifier("darwin-cpu-x86_64"))
  )
)
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")



In [ ]:
import org.tensorflow.framework.GraphDef
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.ops.{Files, Image => TImage}
import org.platanios.tensorflow.api.core.types.UByte
import org.platanios.tensorflow.api.core.client.FeedMap
import java.io.{BufferedInputStream, File, FileInputStream}
import scala.math.sqrt
import scala.math.abs
import smile.stat.distribution.GaussianDistribution
import org.platanios.tensorflow.api.ops.NN._

## Initialization
*Modify path to directories in function of your configuration*

In [ ]:
val basedir = "data/models"
val modelname = "anonymizer"
val modelfilename = "weights_face_v1.0.0.pb"
val modelGraphPath = sys.env.getOrElse("HOME", "/tmp") + s"/${basedir}/${modelname}/${modelfilename}"

val imageFilePath = sys.env("HOME") + "/Downloads/baywatch.png"

val basedir = "data/models"
val modelname = "anonymizer"
val modelfilename = "weights_face_v1.0.0.pb"
val imageFilePath = sys.env("HOME") + "/Desktop/audiScala/xavier/coco01.png"
val modelGraphPath = sys.env.getOrElse("HOME", "/tmp") + s"/Desktop/audiScala/xavier/${modelfilename}"

In [ ]:
lazy val graphDef = GraphDef.parseFrom(
    new BufferedInputStream(new FileInputStream(new File(modelGraphPath))))
lazy val graph = Graph.fromGraphDef(graphDef)

In [ ]:
val session = Session(graph)
val sessionSimple = Session()

## Prepare recuperation of graph results

In [ ]:
val imagePlaceholder = graph.getOutputByName("image_tensor:0").toUByte
val detectionBoxes = graph.getOutputByName("detection_boxes:0").toFloat
val detectionScores = graph.getOutputByName("detection_scores:0").toFloat
val detectionClasses = graph.getOutputByName("detection_classes:0").toFloat
val numDetections = graph.getOutputByName("num_detections:0").toFloat


## Open and transform image

In [ ]:
val (imgTensor, fileNamePlaceholder) = tf.createWith(graph = graph) {
    val fileNamePlaceholder = tf.placeholder[String]()
    val fileTensor = Files.readFile(fileNamePlaceholder)
    val imgTensor = TImage.decodePng(fileTensor, 3)
    (imgTensor, fileNamePlaceholder)
  }

In [ ]:
val file = new File(imageFilePath)
val fileNameTensor = Tensor.fill(Shape())(file.getAbsolutePath())
val feedImg = FeedMap(Map(fileNamePlaceholder -> fileNameTensor))

In [ ]:
val imageOuts: Tensor[UByte] = session.run(fetches = imgTensor, feeds = feedImg)

In [ ]:
val feeds = FeedMap(Map(imagePlaceholder -> imageOuts.slice(NewAxis, ---)))

## Detection of face 
*boxes* = Positions of face detected

*score* = Confidence for each detection

*classes* = Face or plate

*num* = Number of detection

In [ ]:
val Seq(boxes, scores, classes, num) =
      session.run(
        fetches =
          Seq(detectionBoxes, detectionScores, detectionClasses, numDetections),
        feeds = feeds)

In [ ]:
val width = imageOuts.shape(1)
val height = imageOuts.shape(0)
val thereshold = 0.7

## Filter detections to keep only one above our thereshold

In [ ]:
val tabBoxes = for {
    i <- 0 until num(0).scalar.asInstanceOf[Float].toInt
    labelId = classes(0, i).toFloat.scalar.toInt
    box = boxes(0, i).toFloat.entriesIterator.toSeq
    y1 = (box(0) * height).toInt
    x1 = (box(1) * width).toInt
    y2 = (box(2) * height).toInt
    x2 = (box(3) * width).toInt
    labelBox = (y1, x1, y2, x2)
    score = scores(0, i).toFloat.scalar
  } yield (labelId, score, labelBox)

val tabBoxesFiltered = tabBoxes.filter {case (_,y,_) => y > thereshold}
()

## Create a mask to delimitate areas where the blurring need to be apply in the image


In [ ]:
// 0 is corresponding to an area where no blur is necessary and 1 is the opposite
val maskBlur = Array.fill(height, width)(0)

In [ ]:
// Thanks to the box detection, we fill our mask with 1 where we need to apply blurring
tabBoxesFiltered.map {case (x,y,z) => z}
                .foreach {case (x1,y1,x2,y2) => (x1 until x2)
                    .foreach(xi => (y1 until y2)
                        .foreach(yi => maskBlur(xi)(yi) = 1))
}
val maskBlurFlatten = maskBlur.flatten

In [ ]:
val tensorMaskFlatten = Tensor(maskBlurFlatten)
val tensorMaskReshape = tf.reshape(tensorMaskFlatten, Shape(height, width))

## Params for blurring and smoothing

In [ ]:
val channels = 3 // RVB = 3
val smooth_boxes = true // Smooth area around blurring
val kernel_size = 21 // Kernel for blurring
val sigma = 2 // Standard deviation for blurring
val box_kernel_size = 9 // Size for smoothing, need to be > 1

## Generate a 2D gaussian filter...

In [ ]:
val gaussian = new GaussianDistribution(0.0, sigma)

In [ ]:
val interval = (2 * sigma + 1.0) / kernel_size

In [ ]:
val g1dcdf = (-sigma - interval/2.0 to sigma + interval/2.0 by (2.0*sigma + interval)/kernel_size)
    .map(gaussian.cdf(_))

In [ ]:
val k1d = g1dcdf.sliding(2).map { case Seq(x, y) => y - x }.toArray

In [ ]:
val k1dSqrt = for {
    x <- k1d
    y <- k1d
} yield sqrt(x*y)

In [ ]:
val sumK1d = k1dSqrt.sum
val kernel = k1dSqrt.map(x => x/sumK1d)

In [ ]:
val kernelBlurring = tf.reshape(kernel, Shape(kernel_size, kernel_size, 1, 1)).toFloat

## Generate smoothing filter

In [ ]:
val filter_size = List(box_kernel_size, box_kernel_size)
val factor: List[Int] = filter_size.map(size => ((size + 1) / 2).toInt)
val centerX = if (filter_size(0) % 2 == 1) factor(0) - 1 else factor(0) - 0.5
val centerY = if (filter_size(1) % 2 == 1) factor(1) - 1 else factor(1) - 0.5

val vectorX = (0 until filter_size(0)).toList
val vectorY = (0 until filter_size(1)).toList
val kernelTemp = Array.ofDim[Float](filter_size(0), filter_size(1))

(0 until filter_size(0))
    .foreach(i => (0 until filter_size(1))
        .foreach(j => {
            kernelTemp(i)(j) = (1.0f - abs(vectorX(i) - centerX).toFloat / factor(0).toFloat) * (1.0f - abs(vectorY(j) - centerY).toFloat / factor(1).toFloat)
        }
    )
)

val kernelFlat = kernelTemp.flatten
val sumSmoothing = kernelFlat.sum
val kernelSmoothing = Tensor(kernelFlat.map(element => element / sumSmoothing))
val tensorSmoothing = tf.reshape(kernelSmoothing, Shape(filter_size(0), filter_size(1), 1, 1))

## Apply blurring with our gaussian kernel 

### Padding before the convolution to avoid border effect

In [ ]:
val pad = (kernel_size - 1) / 2

In [ ]:
val paddings = Tensor(Tensor(pad, pad), Tensor(pad, pad), Tensor(0, 0))

In [ ]:
val imageWithPadding = (tf.pad(imageOuts, paddings=paddings, mode=tf.ReflectivePadding)).toFloat

#### Convolutions on each channel

We need to do a little trick because the original python code use the function *tf.nn.depthwise_conv2d_native*. 

Unfortunately, this function doesn't exist inside Tensorflow **Scala**.

To replace this function, we split the image between each channel (Red, green and blue).

After that, we use the basic convolution on each channel with our gaussian kernel.

Finally, we re-combined all channel to have the blurred image.

In [ ]:
val tabImages = (0 to 2).toList.map(i => tf.slice(imageWithPadding, Tensor(0,0,i), Tensor(-1,-1,1)).toFloat)
val tabImages4D = tabImages.map(img => img.slice(NewAxis, ---))

In [ ]:
val tensorMaskReshapeFinal = (tensorMaskReshape.slice(NewAxis, ---, NewAxis))

## Convolution to combined gaussian filter and original image

In [ ]:
val tabConvolution = tabImages4D.map(colors => tf.conv2D(input = colors, filter = kernelBlurring, stride1 = 1, stride2 = 1, padding=org.platanios.tensorflow.api.ops.NN.ValidConvPadding))
val imageAfterConvolution = tf.concatenate(inputs = tabConvolution, axis = 3)

## Convolution to smooth the mask

In [ ]:

val smoothedMask = tf.reshape(tf.conv2D(input = tensorMaskReshapeFinal.toFloat, filter = tensorSmoothing, stride1 = 1, stride2 = 1, padding=org.platanios.tensorflow.api.ops.NN.SameConvPadding, name="smooth_mask"), Shape(height, width, 1))


## Combined blurred image and mask to only blur necessary areas

In [ ]:
val reshapeBlur = tf.reshape(imageAfterConvolution, Shape(height, width, channels))
val imageWithoutBox = imageOuts.toFloat * (Tensor(1.0f) - smoothedMask)
val imageCombined = ((reshapeBlur * smoothedMask) + imageWithoutBox).toUByte

val bluredImage = sessionSimple.run(fetches = imageCombined)

In [ ]:
val imgFinal = tf.createWith(graph = graph) {
    val exampleImage = tf.decodeRaw[Byte](tf.image.encodePng(bluredImage))
    session.run(fetches = exampleImage)
}

## Display result

In [ ]:
Image.fromArray(imgFinal.entriesIterator.toArray, Image.PNG, width=Some("500"))